<a href="https://colab.research.google.com/github/kucan23/Dark-Pattern_ML/blob/main/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks


/content/drive/MyDrive/Colab Notebooks


In [ ]:
import numpy as np

import pandas as pd
from plotly.offline import iplot
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk

nltk.download('stopwords')

from keras.regularizers import l2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
file_path = "text_data.tsv"
df = pd.read_csv(file_path, sep='\t', header=0)

target_variable = 'Pattern Category'
data = df["text"]
print(df.shape)

(2356, 4)


In [ ]:
df[target_variable].value_counts()

Not Dark Pattern    1178
Scarcity             418
Social Proof         312
Urgency              210
Misdirection         195
Obstruction           27
Sneaking              12
Forced Action          4
Name: Pattern Category, dtype: int64

Not Dark Pattern    1178
Scarcity             418
Social Proof         312
Urgency              210
Misdirection         195
Obstruction           27
Sneaking              12
Forced Action          4
Name: Pattern Category, dtype: int64

In [ ]:
df["text"] = df["text"].reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)

    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df["text"] = df["text"].apply(clean_text)

In [ ]:

MAX_NB_WORDS = 3000

# MAX_SEQUENCE_LENGTH = 2000
MAX_SEQUENCE_LENGTH = 500

# EMBEDDING_DIM = 200
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df["text"])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 3871 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df["text"].values)

print(X[0:2])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

[[286, 33, 8, 3, 38], [1386, 1387]]
Shape of data tensor: (2356, 500)


In [ ]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# Y = le.fit_transform(df[target_variable])

# print('Shape of label tensor:', Y.shape)
# print(le.classes_)

from sklearn.preprocessing import OneHotEncoder

Y = pd.get_dummies(df[target_variable])
print(Y.shape)

(2356, 8)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1884, 500) (1884, 8)
(472, 500) (472, 8)


In [ ]:
column_sums = np.sum(Y_test, axis=0)

print(column_sums)


column_sums = np.sum(Y_train, axis=0)

print(column_sums)

Forced Action         1
Misdirection         44
Not Dark Pattern    220
Obstruction           6
Scarcity             86
Sneaking              1
Social Proof         71
Urgency              43
dtype: int64
Forced Action         3
Misdirection        151
Not Dark Pattern    958
Obstruction          21
Scarcity            332
Sneaking             11
Social Proof        241
Urgency             167
dtype: int64


In [ ]:
from keras.optimizers import Adam

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
# model.add(Dense(50, activation='relu'))
model.add(LSTM(100, dropout=0.4, recurrent_dropout=0.2))
# model.add(LSTM(100, batch_size=1, return_sequences=True))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])
print(model.summary())



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 100)          300000    
                                                                 
 spatial_dropout1d_4 (Spati  (None, 500, 100)          0         
 alDropout1D)                                                    
                                                                 
 lstm_4 (LSTM)               (None, 100)               80400     
                                                                 
 dense_4 (Dense)             (None, 8)                 808       
                                                                 
Total params: 381208 (1.45 MB)
Trainable params: 381208 (1.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
epochs = 20
batch_size =64
early_stopping = EarlyStopping( monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2, callbacks=[early_stopping])


Epoch 1/20
24/24 [==============================] - 24s 852ms/step - loss: 1.8072 - accuracy: 0.4565 - val_loss: 1.4367 - val_accuracy: 0.5225
Epoch 2/20
24/24 [==============================] - 20s 854ms/step - loss: 1.3913 - accuracy: 0.5050 - val_loss: 1.3622 - val_accuracy: 0.5225
Epoch 3/20
24/24 [==============================] - 20s 819ms/step - loss: 1.3220 - accuracy: 0.5050 - val_loss: 1.2573 - val_accuracy: 0.5225
Epoch 4/20
24/24 [==============================] - 20s 825ms/step - loss: 1.1934 - accuracy: 0.5481 - val_loss: 1.1043 - val_accuracy: 0.6313
Epoch 5/20
24/24 [==============================] - 20s 817ms/step - loss: 0.9772 - accuracy: 0.6722 - val_loss: 0.9070 - val_accuracy: 0.6472
Epoch 6/20
24/24 [==============================] - 21s 880ms/step - loss: 0.7509 - accuracy: 0.7910 - val_loss: 0.6844 - val_accuracy: 0.8408
Epoch 7/20
24/24 [==============================] - 20s 844ms/step - loss: 0.5316 - accuracy: 0.8746 - val_loss: 0.5316 - val_accuracy: 0.8647

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_train)
print(y_pred[0])


59/59 [==============================] - 6s 105ms/step
[5.8725236e-06 4.7144908e-04 9.9725145e-01 6.6637194e-06 1.1414397e-03
 2.1669828e-05 3.9400905e-04 7.0750248e-04]


In [ ]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_t=tf.argmax(Y_train, axis=1)

training_accuracy = accuracy_score(y_t, y_pred_classes)
print("Training Accuracy:", training_accuracy)

Training Accuracy: 0.9697452229299363


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

15/15 [==============================] - 1s 85ms/step - loss: 0.2991 - accuracy: 0.9322
Test set
  Loss: 0.299
  Accuracy: 0.932


In [ ]:
import numpy as np
print(Y_test.shape)

y_r=tf.argmax(Y_test, axis=1)

(472, 8)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_proba = model.predict(X_test)
y_pred_classes = np.argmax(y_proba, axis=1)

# Confusion matrix
confusion_mat = confusion_matrix(y_r, y_pred_classes)
print("Confusion Matrix:")
print(confusion_mat)

# Classification report
print("\nClassification Report:")
print(classification_report(y_r, y_pred_classes))


15/15 [==============================] - 1s 84ms/step
Confusion Matrix:
[[  0   1   0   0   0   0   0   0]
 [  0  31   8   0   1   0   1   3]
 [  0   2 215   0   1   0   1   1]
 [  0   0   0   6   0   0   0   0]
 [  0   0   1   0  85   0   0   0]
 [  0   0   1   0   0   0   0   0]
 [  0   0   5   0   0   0  65   1]
 [  0   0   5   0   0   0   0  38]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.91      0.70      0.79        44
           2       0.91      0.98      0.95       220
           3       1.00      1.00      1.00         6
           4       0.98      0.99      0.98        86
           5       0.00      0.00      0.00         1
           6       0.97      0.92      0.94        71
           7       0.88      0.88      0.88        43

    accuracy                           0.93       472
   macro avg       0.71      0.68      0.69       472
weighted avg       0.93    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
new_text = "No Thanks, I don't like exclusive deals"
new_text = clean_text(new_text)  # Apply preprocessing
tokenized_text = tokenizer.texts_to_sequences([new_text])[0]  # Tokenize
input_data = pad_sequences([tokenized_text], maxlen=MAX_SEQUENCE_LENGTH)  # Pad

prediction = model.predict(input_data)
print(prediction)
predicted_category = prediction.argmax()
print (predicted_category)

1/1 [==============================] - 0s 280ms/step
[[3.7831833e-04 9.8815829e-01 3.4373032e-04 1.1437229e-03 2.5367499e-03
  8.5703097e-04 1.8184625e-03 4.7637220e-03]]
1


In [ ]:


import tensorflow as tf
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('Keras93_c.tflite', 'wb').write(tfmodel)



OSError: Unable to load model. Filepath is not an hdf5 file (or h5py is not available) or SavedModel. Received: filepath=<keras.src.engine.sequential.Sequential object at 0x7bf6340e0910>

In [ ]:
# import tensorflow as tf
# interpreter = tf.lite.Interpreter(model_path="KFINAL_c.tflite")

# interpreter.allocate_tensors()
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

In [ ]:
# new_text = "No Thanks, I don't like exclusive dealS"
# new_text = clean_text(new_text)  # Apply preprocessing
# tokenized_text = tokenizer.texts_to_sequences([new_text])[0]  # Tokenize
# input_data = pad_sequences([tokenized_text], maxlen=MAX_SEQUENCE_LENGTH)  # Pad


In [ ]:
# input_data = input_data.astype('float32')  # Convert to FLOAT32
# interpreter.set_tensor(input_details[0]['index'], input_data)

# interpreter.invoke()
# output_data = interpreter.get_tensor(output_details[0]['index'])
# predicted_category = output_data.argmax()  # Get the index of the predicted category
# print (output_data)
# print(predicted_category)